In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# إعدادات أساسية
img_size = 96
categories = ['close eyes', 'open eyes']

# مسارات البيانات
base_dir = '/kaggle/input/mrl-eye-dataset/data'
train_dir = os.path.join(base_dir, 'train')

def load_data(data_dir):
    data = []
    labels = []

    for category in categories:
        folder = os.path.join(data_dir, category)
        label = categories.index(category)  # 0 = مغلقة، 1 = مفتوحة

        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # -> Grayscale 
                img = cv2.resize(img, (img_size, img_size))
                data.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")


    # convert to numpy arrays
    data = np.array(data).reshape(-1, img_size, img_size, 1) / 255.0
    # 1 -> Grayscale -> one channel
    # -1 -> number of images -> (no need to specify it)
    # 255 -> Normalization(0-1)

    labels = np.array(labels)
    
    return shuffle(data, labels, random_state=42) # shuffle the data and make each data correspond to its label

# تحميل البيانات من مجلد التدريب
X, y = load_data(train_dir)

# تقسيم البيانات إلى تدريب واختبار بنسبة 80-20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# عرض الأبعاد للتأكد من النسب
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)


Training data shape: (65340, 96, 96, 1)
Test data shape: (16335, 96, 96, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # 96 * 96 * 1 
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 1)), # 96 - 3 + 1 = 94
    layers.MaxPooling2D(2, 2), # 94 / 2 = 47

    layers.Conv2D(64, (3,3), activation='relu'),    # 47 - 3 + 1 = 45
    layers.MaxPooling2D(2, 2),  # 45 / 2 = 22

    layers.Conv2D(128, (3,3), activation='relu'),   # 22 - 3 + 1 = 20
    layers.MaxPooling2D(2, 2), # 20 / 2 = 10

    layers.Flatten(), # convert to 1D array => 10 * 10 * 128 = 12800
    layers.Dense(128, activation='relu'),  # 128 neurons to learn features (every neuron learns a feature)
    layers.Dropout(0.5), # 50% dropout
    layers.Dense(1, activation='sigmoid') # 1 neuron for binary classification
])

model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

model.summary()


2025-04-22 22:26:12.304861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745360772.740029      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745360772.845692      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-22 22:26:31.613482: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 94, 94, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 47, 47, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 45, 45, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 20, 20, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,638,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,731,329 (6.60 MB)

 Trainable params: 1,731,329 (6.60 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test),
                    batch_size=32)

# accuracy, loss -> train
# val_accuracy, val_loss -> test

Epoch 1/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 596s 290ms/step - accuracy: 0.8781 - loss: 0.2738 - val_accuracy: 0.9638 - val_loss: 0.0984
Epoch 2/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 608s 298ms/step - accuracy: 0.9695 - loss: 0.0870 - val_accuracy: 0.9818 - val_loss: 0.0540
Epoch 3/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 600s 294ms/step - accuracy: 0.9774 - loss: 0.0670 - val_accuracy: 0.9804 - val_loss: 0.0547
Epoch 4/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 594s 291ms/step - accuracy: 0.9812 - loss: 0.0523 - val_accuracy: 0.9769 - val_loss: 0.0677
Epoch 5/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 594s 291ms/step - accuracy: 0.9838 - loss: 0.0458 - val_accuracy: 0.9857 - val_loss: 0.0405
Epoch 6/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 620s 290ms/step - accuracy: 0.9854 - loss: 0.0404 - val_accuracy: 0.9870 - val_loss: 0.0383
Epoch 7/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 590s 289ms/step - accuracy: 0.9875 - loss: 0.0361 - val_accuracy: 0.9854 - val_loss: 0.0387
Epoch 8/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 590s 289ms/step - ac

In [4]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


511/511 ━━━━━━━━━━━━━━━━━━━━ 36s 70ms/step - accuracy: 0.9874 - loss: 0.0343
Test Accuracy: 98.82%


In [ ]:
model.save("eye_state_model.h5")
print("✅ تم حفظ الموديل بنجاح!")


✅ تم حفظ الموديل بنجاح!
